<a href="https://colab.research.google.com/github/newmantic/Johnson/blob/main/Johnson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import heapq
from collections import defaultdict

class JohnsonAlgorithm:
    def __init__(self, graph):
        """
        Initialize the Johnson's Algorithm.
        :param graph: A dictionary where the keys are nodes and the values are dictionaries
                      of neighbors with the weight of the edge as the value.
                      For example: {0: {1: 3, 2: 8}, 1: {2: 2, 3: -4}, ...}
        """
        self.graph = graph
        self.V = len(graph)
        self.distances = {}
        self.modified_graph = defaultdict(dict)

    def bellman_ford(self, source):
        """
        Bellman-Ford algorithm to detect negative weight cycles and calculate shortest paths.
        :param source: The source vertex.
        :return: A list of distances from the source to all vertices or False if a negative cycle is detected.
        """
        dist = {v: float('inf') for v in range(self.V + 1)}
        dist[source] = 0

        for _ in range(self.V):
            for u in self.graph:
                for v in self.graph[u]:
                    if dist[u] + self.graph[u][v] < dist[v]:
                        dist[v] = dist[u] + self.graph[u][v]

        # Check for negative weight cycles
        for u in self.graph:
            for v in self.graph[u]:
                if dist[u] + self.graph[u][v] < dist[v]:
                    return False  # Negative weight cycle detected

        return dist

    def dijkstra(self, source):
        """
        Dijkstra's algorithm to find the shortest paths from the source to all vertices.
        :param source: The source vertex.
        :return: A dictionary of shortest paths from the source to all vertices.
        """
        dist = {v: float('inf') for v in range(self.V)}
        dist[source] = 0
        priority_queue = [(0, source)]

        while priority_queue:
            current_dist, u = heapq.heappop(priority_queue)

            if current_dist > dist[u]:
                continue

            for v in self.modified_graph[u]:
                alt = dist[u] + self.modified_graph[u][v]
                if alt < dist[v]:
                    dist[v] = alt
                    heapq.heappush(priority_queue, (alt, v))

        return dist

    def johnson(self):
        """
        Johnson's algorithm to find all-pairs shortest paths in a sparse graph with possible negative weights.
        :return: A dictionary of dictionaries where the keys are vertex pairs and the values are the shortest path distances.
        """
        # Add a new vertex s and connect it to all vertices with edge weight 0
        self.graph[self.V] = {v: 0 for v in range(self.V)}

        # Step 1: Run Bellman-Ford from the new vertex to find potential function h(v)
        h = self.bellman_ford(self.V)
        if h is False:
            return "Negative weight cycle detected"

        # Step 2: Reweight all edges to eliminate negative weights
        for u in self.graph:
            for v in self.graph[u]:
                if u != self.V:
                    self.modified_graph[u][v] = self.graph[u][v] + h[u] - h[v]

        # Step 3: Run Dijkstra for each vertex
        for u in range(self.V):
            self.distances[u] = self.dijkstra(u)

        # Step 4: Restore original edge weights
        for u in self.distances:
            for v in self.distances[u]:
                if self.distances[u][v] != float('inf'):
                    self.distances[u][v] = self.distances[u][v] - h[u] + h[v]

        return self.distances




In [4]:
def test_case_1():
    graph = {
        0: {1: 4, 2: 8},
        1: {2: -2, 3: 5},
        2: {3: 3},
        3: {}
    }
    ja = JohnsonAlgorithm(graph)
    all_pairs_shortest_paths = ja.johnson()
    print("All-Pairs Shortest Paths:")
    for u in all_pairs_shortest_paths:
        for v in all_pairs_shortest_paths[u]:
            print(f"Shortest path from {u} to {v}: {all_pairs_shortest_paths[u][v]}")

test_case_1()

All-Pairs Shortest Paths:
Shortest path from 0 to 0: 0
Shortest path from 0 to 1: 4
Shortest path from 0 to 2: 2
Shortest path from 0 to 3: 5
Shortest path from 1 to 0: inf
Shortest path from 1 to 1: 0
Shortest path from 1 to 2: -2
Shortest path from 1 to 3: 1
Shortest path from 2 to 0: inf
Shortest path from 2 to 1: inf
Shortest path from 2 to 2: 0
Shortest path from 2 to 3: 3
Shortest path from 3 to 0: inf
Shortest path from 3 to 1: inf
Shortest path from 3 to 2: inf
Shortest path from 3 to 3: 0


In [5]:
def test_case_2():
    graph = {
        0: {1: 10, 2: 5},
        1: {2: 2, 3: 1},
        2: {1: 3, 3: 9, 4: 2},
        3: {4: 4},
        4: {3: 6}
    }
    ja = JohnsonAlgorithm(graph)
    all_pairs_shortest_paths = ja.johnson()
    print("All-Pairs Shortest Paths:")
    for u in all_pairs_shortest_paths:
        for v in all_pairs_shortest_paths[u]:
            print(f"Shortest path from {u} to {v}: {all_pairs_shortest_paths[u][v]}")

test_case_2()

All-Pairs Shortest Paths:
Shortest path from 0 to 0: 0
Shortest path from 0 to 1: 8
Shortest path from 0 to 2: 5
Shortest path from 0 to 3: 9
Shortest path from 0 to 4: 7
Shortest path from 1 to 0: inf
Shortest path from 1 to 1: 0
Shortest path from 1 to 2: 2
Shortest path from 1 to 3: 1
Shortest path from 1 to 4: 4
Shortest path from 2 to 0: inf
Shortest path from 2 to 1: 3
Shortest path from 2 to 2: 0
Shortest path from 2 to 3: 4
Shortest path from 2 to 4: 2
Shortest path from 3 to 0: inf
Shortest path from 3 to 1: inf
Shortest path from 3 to 2: inf
Shortest path from 3 to 3: 0
Shortest path from 3 to 4: 4
Shortest path from 4 to 0: inf
Shortest path from 4 to 1: inf
Shortest path from 4 to 2: inf
Shortest path from 4 to 3: 6
Shortest path from 4 to 4: 0


In [6]:
def test_case_3():
    graph = {
        0: {1: 1},
        1: {2: 1},
        2: {3: 1},
        3: {1: -3}
    }
    ja = JohnsonAlgorithm(graph)
    result = ja.johnson()
    print(result)  # Expected: "Negative weight cycle detected"

test_case_3()

Negative weight cycle detected
